# Classificação de Texto e Análise de Sentimentos

In [1]:
import pandas as pd
import time

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 100)

# exemplo de documento e corpus
df_fiap = pd.DataFrame({
    'text': [
      'Sobre postech? Eu gostei muito do postech da FIAP',
      'O postech da FIAP pode melhorar, não gostei muito',
      'Foi muito importante para meu desenvolvimento',
      'Poderia ser mais técnico. Não gostei'
    ],
    'class': [
      'positivo',
      'negativo',
      'positivo',
      'negativo'
    ]})

df_fiap.head()

,text,class
0,Sobre postech? Eu gostei muito do postech da FIAP,positivo
1,"O postech da FIAP pode melhorar, não gostei muito",negativo
2,Foi muito importante para meu desenvolvimento,positivo
3,Poderia ser mais técnico. Não gostei,negativo


### BoW - Bag Of Words

In [2]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(ngram_range=(1,1))
vect.fit(df_fiap.text)
text_vect = vect.transform(df_fiap.text)

print(pd.DataFrame(text_vect.A, columns=vect.get_feature_names_out()).T.to_string())

                 0  1  2  3
da               1  1  0  0
desenvolvimento  0  0  1  0
do               1  0  0  0
eu               1  0  0  0
fiap             1  1  0  0
foi              0  0  1  0
gostei           1  1  0  1
importante       0  0  1  0
mais             0  0  0  1
melhorar         0  1  0  0
meu              0  0  1  0
muito            1  1  1  0
não              0  1  0  1
para             0  0  1  0
pode             0  1  0  0
poderia          0  0  0  1
postech          2  1  0  0
ser              0  0  0  1
sobre            1  0  0  0
técnico          0  0  0  1


### TF-DF

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(ngram_range=(1,1), use_idf=True)
vect.fit(df_fiap.text)
text_vect = vect.transform(df_fiap.text)

print(pd.DataFrame(text_vect.A, columns=vect.get_feature_names_out()).T.to_string())

                        0         1         2         3
da               0.287039  0.342426  0.000000  0.000000
desenvolvimento  0.000000  0.000000  0.430037  0.000000
do               0.364073  0.000000  0.000000  0.000000
eu               0.364073  0.000000  0.000000  0.000000
fiap             0.287039  0.342426  0.000000  0.000000
foi              0.000000  0.000000  0.430037  0.000000
gostei           0.232383  0.277223  0.000000  0.284626
importante       0.000000  0.000000  0.430037  0.000000
mais             0.000000  0.000000  0.000000  0.445922
melhorar         0.000000  0.434323  0.000000  0.000000
meu              0.000000  0.000000  0.430037  0.000000
muito            0.232383  0.277223  0.274487  0.000000
não              0.000000  0.342426  0.000000  0.351570
para             0.000000  0.000000  0.430037  0.000000
pode             0.000000  0.434323  0.000000  0.000000
poderia          0.000000  0.000000  0.000000  0.445922
postech          0.574078  0.342426  0.000000  0

# NLP

In [4]:
import nltk
import re
import string
import unicodedata

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [5]:
# função para normalizar caracteres especiais - acentos, ç e etc 
def normalize_accents(text):
    return unicodedata.normalize('NFKD', text.encode('ASCII', 'ignore').decode('utf-8'))

In [6]:
# funçao para remover a pontuação do texto
def remove_punctuation(text):
    pontuacao = string.punctuation # todas as pontuações existentes
    table = str.maketrans({key: " " for key in pontuacao}) # cria um dicionário com as pontuações existentes substituindo-as por um espaço em branco
    text = text.translate(table)
    return text 

In [7]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
# função de normalização do texto 
def normalize_str(text):
    text = text.lower() #transforma todas as letras em minúsculas
    text = remove_punctuation(text)
    text = normalize_accents(text)
    text = re.sub(re.compile(r" +"), " ", text) #elimina espaços em branco existentes no texto
    return " ".join([w for w in text.split()]) #no retorno, a string é reorganizada novamente, eliminando qualquer espaço que possa ter sido gerado pelas etapas anteriores.

In [9]:
# função de tokenização do texto 
def tokenizer(text):
    stop_words = nltk.corpus.stopwords.words("english") #selecione a língua da base
    if isinstance(text, str):
        text = normalize_str(text)
        text = "".join([w for w in text if not w.isdigit()]) #remove qualquer número do texto
        text = word_tokenize(text)
        text = [x for x in text if x not in stop_words] #remove stopwords do texto
        text = [y for y in text if len(y) > 2] #opcional - remove qualquer palavra com menos de 2 caracteres
        return " ".join([t for t in text])
    else:
        return None

In [10]:
tokenizer("***Exemplo$ de 12 normalização!%()")

'exemplo normalizao'

In [11]:
df = pd.read_csv('dados/uci-news-aggregator.csv')
df = df[['TITLE', 'CATEGORY']]
df.head(3)

,TITLE,CATEGORY
0,"Fed official says weak data caused by weather,...",b
1,Fed's Charles Plosser sees high bar for change...,b
2,US open: Stocks fall after Fed official hints ...,b


In [12]:
#categories: b = business, t = science and technology, e = entertainment, m = health

In [13]:
from sklearn.utils import shuffle

#embaralhando os dados
df = shuffle(df)
df = df.reset_index(drop=True)
df.head(3)

,TITLE,CATEGORY
0,Decrease in average gas price is slight but co...,t
1,Eat and Drink Your Tax Day Woes Away With Thes...,b
2,The fashion world's winners at this year's Web...,e


In [14]:
df['Title_treated'] = df['TITLE'].apply(tokenizer)
df.head(3)

,TITLE,CATEGORY,Title_treated
0,Decrease in average gas price is slight but co...,t,decrease average gas price slight could signal...
1,Eat and Drink Your Tax Day Woes Away With Thes...,b,eat drink tax day woes away offers
2,The fashion world's winners at this year's Web...,e,fashion world winners year webby awards


In [15]:
from sklearn.model_selection import train_test_split

X = df['Title_treated'].values # .values - retorna os valores dessa coluna em forma de array
y = df['CATEGORY'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [16]:
X_train

array(['market update nasdaq aapl advanced technologies apple',
       'ecb measures expected coming months',
       'kanye west booed wireless festival video shows rant led', ...,
       'argentina default may one', 'train dragon',
       'japan australia deal makes tpp even important'], dtype=object)

In [17]:
# criando uma representação utilizando o BoW
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(lowercase=False)
vect.fit(X_train)
X_train = vect.transform(X_train)
X_train

<295693x43896 sparse matrix of type '<class 'numpy.int64'>'
	with 1955923 stored elements in Compressed Sparse Row format>

In [18]:
'''O retorno foi uma matriz esparsa. 
O primeiro valor é a quantidade de amostras que eu tenho no conjunto de treino. 
O segundo valor é o tamanho do dicionário que foi aprendido durante o processo de treino.'''

'O retorno foi uma matriz esparsa. \nO primeiro valor é a quantidade de amostras que eu tenho no conjunto de treino. \nO segundo valor é o tamanho do dicionário que foi aprendido durante o processo de treino.'

In [ ]:
X_test = vect.transform(X_test)
X_test

<126726x44067 sparse matrix of type '<class 'numpy.int64'>'
	with 831472 stored elements in Compressed Sparse Row format>

In [ ]:
# treinando com SVM 
from sklearn import svm

clf = svm.SVC(kernel='linear')
start_time = time.time()
clf.fit(X_train, y_train)
end_time = time.time()
print('tempo decorrido: ',end_time-start_time, 'segundos')
y_pred = clf.predict(X_test)

tempo decorrido:  7766.649791955948 segundos


In [ ]:
# tempo decorrido para treino 
import datetime 
sec = end_time - start_time
print(str(datetime.timedelta(seconds = sec)))

2:09:26.649792


In [ ]:
from sklearn import metrics
print('Accuracy:', metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9427031548379969


# Análise de Sentimentos

In [19]:
df = pd.read_csv("dados/tweets_classificados.csv")
df = df[['texto', 'sentimento']]
df.head(3)

,texto,sentimento
0,���⛪ @ Catedral de Santo Antônio - Governador ...,Neutro
1,"� @ Governador Valadares, Minas Gerais https:/...",Neutro
2,"�� @ Governador Valadares, Minas Gerais https:...",Neutro


In [20]:
df.sentimento.unique() # tipos de sentimento no texto

array(['Neutro', 'Negativo', 'Positivo'], dtype=object)

In [21]:
# shuffle nos dados
df = shuffle(df)
df = df.reset_index(drop=True)
df.head(3)

,texto,sentimento
0,@Vicente_Q_Filho @OnlyKetanAmr @c0nvey Esses R...,Negativo
1,"Em calamidade financeira, governo de MG compra...",Negativo
2,"RT @Larry1906: Esse Secretário de Temer, o imb...",Negativo


In [24]:
# função de tokenização do texto agora com o português
def tokenizer(text):
    stop_words = nltk.corpus.stopwords.words("portuguese") #selecione a língua da base
    if isinstance(text, str):
        text = normalize_str(text)
        text = "".join([w for w in text if not w.isdigit()]) #remove qualquer número do texto
        text = word_tokenize(text)
        text = [x for x in text if x not in stop_words] #remove stopwords do texto
        text = [y for y in text if len(y) > 2] #opcional - remove qualquer palavra com menos de 2 caracteres
        return " ".join([t for t in text])
    else:
        return None

In [25]:
df['tweet_tratado'] = df['texto'].apply(tokenizer)
df.head(3)

,texto,sentimento,tweet_tratado
0,@Vicente_Q_Filho @OnlyKetanAmr @c0nvey Esses R...,Negativo,vicente filho onlyketanamr cnvey milhes poderi...
1,"Em calamidade financeira, governo de MG compra...",Negativo,calamidade financeira governo compra dois heli...
2,"RT @Larry1906: Esse Secretário de Temer, o imb...",Negativo,larry secretrio temer imbecil bruno jlio pmdb ...


In [26]:
# split
X = df['tweet_tratado'].values
y = df['sentimento'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [28]:
# CountVectorizer

vect = CountVectorizer(lowercase=False)
vect.fit(X_train)
X_train = vect.transform(X_train)
X_train

<4042x9707 sparse matrix of type '<class 'numpy.int64'>'
	with 42296 stored elements in Compressed Sparse Row format>

In [29]:
X_test = vect.transform(X_test)
X_test

<1733x9707 sparse matrix of type '<class 'numpy.int64'>'
	with 14993 stored elements in Compressed Sparse Row format>

In [31]:
#treinando com SVM
from sklearn import svm

clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [37]:
from sklearn import metrics
print('Accuracy: ',(metrics.accuracy_score(y_test, y_pred)*100).round(2),'%')

Accuracy:  95.33 %
